# Plots of Trade-offs

In this notebook we compare different time discretization methods. First, we collect
a trajectory data from the environment at a fine discretization level (this is also
the discretization level we run the policy at -- right now, anyway). Then we compare:

1. Using uniform discretization at different granularities, e.g. updating with every
    1st, 10th, 100th, ...? interactions.
2. Using the adaptive method with different tolarances.

In order to average out randomness, we'll repeat each setting 3 times for now.

In [12]:
import gymnasium as gym
from adaptive_time.features import Fourier_Features
import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import random

import adaptive_time.utils
from adaptive_time import environments
from adaptive_time import mc2
from adaptive_time import samplers

seed = 13

In [2]:
gym.register(
    id="CartPole-OURS-v0",
    entry_point="adaptive_time.environments.cartpole:CartPoleEnv",
    vector_entry_point="adaptive_time.environments.cartpole:CartPoleVectorEnv",
    max_episode_steps=500,
    reward_threshold=475.0,
)

def reset_randomness(seed, env):
    random.seed(seed)
    np.random.seed(seed)
    # env.seed(seed)
    env.action_space.seed(seed)

In [3]:
# Sample usage of the environment.
print(
    "We run the same environment and simple policy twice,\n"
    "with different time discretizations. The policy we use\n"
    "will always go left, so the time discretization does not\n"
    "make a difference to the behaviour, and the total return\n"
    "will be the same.")
print()

policy = lambda obs: 0

env = gym.make('CartPole-OURS-v0')
tau = 0.02
env.stepTime(tau)

reset_randomness(seed, env)
traj = environments.generate_trajectory(env, seed, policy)
total_return_1 = sum(ts[2] for ts in traj)
print("Total undiscounted return: ", total_return_1)

env = gym.make('CartPole-OURS-v0')
tau = 0.002
env.stepTime(tau)

reset_randomness(seed, env)
traj = environments.generate_trajectory(env, seed, policy)
total_return_2 = sum(ts[2] for ts in traj)
print("Total undiscounted return: ", total_return_2)

np.testing.assert_almost_equal(total_return_1, total_return_2, decimal=0)

print()
print(
    "We can expect some difference because we may get an extra\n"
    "timesteps in the more fine-grained discretization, but the\n"
    "difference should be smallish.")

We run the same environment and simple policy twice,
with different time discretizations. The policy we use
will always go left, so the time discretization does not
make a difference to the behaviour, and the total return
will be the same.

Total undiscounted return:  10.589912009424973
Total undiscounted return:  10.017508472458736

We can expect some difference because we may get an extra
timesteps in the more fine-grained discretization, but the
difference should be smallish.


/Users/szepi1991/Code/adaptive_time/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.stepTime to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.stepTime` for environment variables or `env.get_wrapper_attr('stepTime')` that will search the reminding wrappers.
  logger.warn(


**NOTE** you must adjust the discount factor if changing time-scales!

In [4]:
phi = Fourier_Features()
phi.init_fourier_features(4,4)
x_thres = 4.8
theta_thres = 0.418
phi.init_state_normalizers(np.array([x_thres,2.0,theta_thres,1]), np.array([-x_thres,-2.0,-theta_thres,-1]))
phi.num_parameters

625

In [9]:

def run_experiment(
        seed, env, sampler, epsilon, num_episodes, gamma, print_trajectory=False):
    """Returns the number of episodes it took to solve the environment."""

    # We record:
    returns_per_episode_q = np.zeros((2, num_episodes))
    average_returns_q = np.zeros((2, num_episodes))  # the cumulative average of the above
    predicted_returns_q = np.zeros((2, num_episodes))

    reset_randomness(seed, env)

    observation, _ = env.reset(seed=seed)
    d = len(phi.get_fourier_feature(observation))
    assert d == phi.num_parameters
    features = np.identity(2 * d)   # An estimate of A = xx^T
    targets = np.zeros(2 * d)  # An estimate of b = xG
    weights = np.zeros(2 * d)   # The weights that approximate A^{-1} b

    x_0 = phi.get_fourier_feature([0,0,0,0])  # the initial state
    x_sa0 = mc2.phi_sa(x_0, 0)
    x_sa1 = mc2.phi_sa(x_0, 1)

    for episode in range(num_episodes):
        def policy(state):
            if random.random() < epsilon:
                return env.action_space.sample()
            # Otherwise calculate the best action.
            x = phi.get_fourier_feature(state)
            qs = np.zeros(2)
            for action in [0, 1]:
                x_sa = mc2.phi_sa(x, action)
                qs[action] = np.inner(x_sa.flatten(), weights)
            # adaptive_time.utils.softmax(qs, 1)
            return adaptive_time.utils.argmax(qs)

        trajectory = environments.generate_trajectory(env, policy=policy, max_steps=100_000)
        if trajectory is None:
            print("episode:", episode)
            print("Did not drop it for a long time, returning!")
            return episode

        if print_trajectory:
            print("trajectory-len: ", len(trajectory), "; trajectory:")
            for idx, (o, a, r, o_) in enumerate(trajectory):
                # * ignore reward, as it is always the same here.
                # * o_ is the same as the next o.
                print(f"* {idx:4d}: o: {o}\n\t --> action: {a}")

        weights, targets, features, cur_avr_returns = mc2.ols_monte_carlo(
            trajectory, sampler, tqdm, phi, weights, targets, features, x_0, gamma)
        
        # Store the empirical and predicted returns. For any episode, we may
        # or may not have empirical returns for both actions. When we don't have an
        # estimate, `nan` is returned.
        returns_per_episode_q[:, episode] = cur_avr_returns
        average_returns_q[:, episode] = np.nanmean(returns_per_episode_q[:, :episode+1], axis=1)

        predicted_returns_q[0, episode] = np.inner(x_sa0.flatten(), weights)
        predicted_returns_q[1, episode] = np.inner(x_sa1.flatten(), weights)
        print(
            'episode:', episode,
            ' empirical returns:' , returns_per_episode_q[:, episode],
            ' predicted returns:' , predicted_returns_q[:, episode])
    
    return -1

In [13]:
num_episodes = 100
epsilon = 0.1

tau = 0.002
env.stepTime(tau)

# sampler = samplers.AdaptiveQuadratureSampler2(tolerance=0.1)
# sampler = samplers.AdaptiveQuadratureSampler2(tolerance=0.0)

samplers_tried = dict(
    q0_3=samplers.AdaptiveQuadratureSampler2(tolerance=0.3),
    q0_1=samplers.AdaptiveQuadratureSampler2(tolerance=0.1),
    q0_03=samplers.AdaptiveQuadratureSampler2(tolerance=0.03),
    q0_0=samplers.AdaptiveQuadratureSampler2(tolerance=0.0),
    u1=samplers.UniformSampler2(1),
    u5=samplers.UniformSampler2(5),
    u10=samplers.UniformSampler2(10),
    u20=samplers.UniformSampler2(20),
)

results = {}
for name, sampler in samplers_tried.items():
    results[name] = run_experiment(seed, env, sampler, epsilon, num_episodes, gamma=0.999)

print()
print("Results:")
for name, num_episodes in results.items():
    print(f"{name}: {num_episodes}")



Using 29/548 samples.


/Users/szepi1991/Code/adaptive_time/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.stepTime to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.stepTime` for environment variables or `env.get_wrapper_attr('stepTime')` that will search the reminding wrappers.
  logger.warn(


  0%|          | 0/548 [00:00<?, ?it/s]

episode: 0  empirical returns: [39.07399847  0.        ]  predicted returns: [38.64580775 12.42227583]
Did 5000 steps! 5000
Did 5000 steps! 10000
Did 5000 steps! 15000
Did 5000 steps! 20000
Did 5000 steps! 25000
Did 5000 steps! 30000
Did 5000 steps! 35000
Did 5000 steps! 40000
Did 5000 steps! 45000
Did 5000 steps! 50000
Did 5000 steps! 55000
Did 5000 steps! 60000
Did 5000 steps! 65000
Did 5000 steps! 70000
Did 5000 steps! 75000
Did 5000 steps! 80000
Did 5000 steps! 85000
Did 5000 steps! 90000
Did 5000 steps! 95000
Did 5000 steps! 100000
Max steps reached! 100001
episode: 1
Did not drop it for a long time, returning!
Using 41/548 samples.


  0%|          | 0/548 [00:00<?, ?it/s]

...

episode: 63  empirical returns: [18.47281975  0.        ]  predicted returns: [17.81406088 16.72096901]
Using 13/254 samples.


  0%|          | 0/254 [00:00<?, ?it/s]

episode: 64  empirical returns: [18.53809636  0.        ]  predicted returns: [17.82634199 16.73325497]
Using 14/264 samples.


  0%|          | 0/264 [00:00<?, ?it/s]

episode: 65  empirical returns: [19.16944421  0.        ]  predicted returns: [17.84823634 16.75543465]
Did 5000 steps! 5000
Did 5000 steps! 10000
Did 5000 steps! 15000
Did 5000 steps! 20000
Did 5000 steps! 25000
Did 5000 steps! 30000
Did 5000 steps! 35000
Did 5000 steps! 40000
Did 5000 steps! 45000
Did 5000 steps! 50000
Did 5000 steps! 55000
Did 5000 steps! 60000
Did 5000 steps! 65000
Did 5000 steps! 70000
Did 5000 steps! 75000
Did 5000 steps! 80000
Did 5000 steps! 85000
Did 5000 steps! 90000
Did 5000 steps! 95000
Did 5000 steps! 100000
Max steps reached! 100001
episode: 66
Did not drop it for a long time, returning!

Results:
q0_3: 1
q0_1: 7
q0_03: 24
q0_0: 24
u1: 24
u5: 4
u10: 31
u20: 66


In [14]:
print()
print("Results:")
for name, num_episodes in results.items():
    print(f"{name}: {num_episodes}")



Results:
q0_3: 1
q0_1: 7
q0_03: 24
q0_0: 24
u1: 24
u5: 4
u10: 31
u20: 66
